This lab on Cross-Validation is a python adaptation of p. 190-194 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. Written by R. Jordan Crouser at Smith College for SDS293: Machine Learning (Spring 2016) and is inspired from  http://www.science.smith.edu/~jcrouser/SDS293/labs/lab7-py.html

# Cross Validation

In [ ]:
import pandas as pd
import numpy as np
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt

# Validation Set approch 

## Data exploration

In [ ]:
auto_data = pd.read_csv('auto.csv', na_values='?').dropna()
auto_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cylinders     392 non-null    int64  
 1   displacement  392 non-null    float64
 2   horsepower    392 non-null    float64
 3   weight        392 non-null    float64
 4   acceleration  392 non-null    float64
 5   model_year    392 non-null    int64  
 6   origin        392 non-null    int64  
 7   mpg           392 non-null    float64
dtypes: float64(5), int64(3)
memory usage: 24.6 KB


In [ ]:
auto_data.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,mpg
0,8,307.0,130.0,3504.0,12.0,70,1,18.0
1,8,350.0,165.0,3693.0,11.5,70,1,15.0
2,8,318.0,150.0,3436.0,11.0,70,1,18.0
3,8,304.0,150.0,3433.0,12.0,70,1,16.0
4,8,302.0,140.0,3449.0,10.5,70,1,17.0


In [ ]:
auto_data.describe()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,mpg
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592,1.576531,23.445918
std,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737,0.805518,7.805007
min,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000,9.000000
25%,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000,1.000000,17.000000
50%,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000,1.000000,22.750000
75%,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000,2.000000,29.000000
max,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000,46.600000


## Data preperation and modeling

### Trail 1

In [ ]:
train_df = auto_data.sample(196, random_state = 1) # split the data into two halfs # random_state to get the same result every time

#print(train_df.head())
#print(train_df.shape)
val_df = auto_data[~auto_data.isin(train_df)].dropna(how = 'all')
#print(val_df.head())
#print(len(val_df))
# For simplisity we will use the only parameter horsepower to estimate the mpg
X_train = train_df['horsepower'].values.reshape(-1,1)
y_train = train_df['mpg']
X_val = val_df['horsepower'].values.reshape(-1,1)
y_val = val_df['mpg']

     cylinders  displacement  horsepower  weight  acceleration  model_year  \
81           4         120.0        97.0  2506.0          14.5          72   
165          4          97.0        75.0  2171.0          16.0          75   
351          4         108.0        75.0  2350.0          16.8          81   
119          4         121.0       112.0  2868.0          15.5          73   
379          4          91.0        67.0  1995.0          16.2          82   

     origin   mpg  
81        3  23.0  
165       3  29.0  
351       3  32.4  
119       2  19.0  
379       3  38.0   (196, 8)


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

# Linear
lm = skl_lm.LinearRegression()

model = lm.fit(X_train, y_train)

print(mean_squared_error(y_val, model.predict(X_val)))

# Quadratic
poly = PolynomialFeatures(degree=2)
X_train2 = poly.fit_transform(X_train)

X_val2 = poly.fit_transform(X_val)

model = lm.fit(X_train2, y_train)
print(mean_squared_error(y_val, model.predict(X_val2)))

# Cubic
poly = PolynomialFeatures(degree=3)
X_train3 = poly.fit_transform(X_train)

X_val3 = poly.fit_transform(X_val)

model = lm.fit(X_train3, y_train)


print(X_train.shape,X_train[0],X_val[0])
print(X_train2.shape,X_train2[0],X_val2[0])
print(X_train3.shape,X_train3[0],X_val3[0])
print(mean_squared_error(y_val, model.predict(X_val3)))

23.361902892587224
20.252690858346554
(196, 1) [97.] [165.]
(196, 3) [1.000e+00 9.700e+01 9.409e+03] [1.0000e+00 1.6500e+02 2.7225e+04]
(196, 4) [1.00000e+00 9.70000e+01 9.40900e+03 9.12673e+05] [1.000000e+00 1.650000e+02 2.722500e+04 4.492125e+06]
20.32560936597247


### Trail 2

In [ ]:
train_df = auto_data.sample(196, random_state = 2) # split the data into two halfs # random_state to get the same result every time
print(train_df.shape,train_df.head())
val_df = auto_data[~auto_data.isin(train_df)].dropna(how = 'all')

# For simplisity we will use the only parameter horsepower to estimate the mpg
X_train = train_df['horsepower'].values.reshape(-1,1)
y_train = train_df['mpg']
X_val = val_df['horsepower'].values.reshape(-1,1)
y_val = val_df['mpg']

model = lm.fit(X_train, y_train)
print(mean_squared_error(y_val, model.predict(X_val)))

# Quadratic
poly = PolynomialFeatures(degree=2)
X_train2 = poly.fit_transform(X_train)
X_val2 = poly.fit_transform(X_val)

model = lm.fit(X_train2, y_train)
print(mean_squared_error(y_val, model.predict(X_val2)))

# Cubic
poly = PolynomialFeatures(degree=3)
X_train3 = poly.fit_transform(X_train)
X_val3 = poly.fit_transform(X_val)

model = lm.fit(X_train3, y_train)
print(mean_squared_error(y_val, model.predict(X_val3)))

(196, 8)      cylinders  displacement  horsepower  weight  acceleration  model_year  \
55           4          91.0        70.0  1955.0          20.5          71   
70           3          70.0        97.0  2330.0          13.5          72   
313          4         151.0        90.0  3003.0          20.1          80   
179          4          91.0        53.0  1795.0          17.5          75   
307          4          98.0        76.0  2144.0          14.7          80   

     origin   mpg  
55        1  26.0  
70        3  19.0  
313       1  24.3  
179       3  33.0  
307       2  41.5  
25.10853905288967
19.722533470492426
19.921367860072696


# Leave-One-Out Cross-Validation (LOOCV)

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

X = auto_data['horsepower'].values.reshape(-1,1)
y = auto_data['mpg'].values.reshape(-1,1)
print(X.shape)
# making n_splits = # of entries make the cross_val_score use only one entry for testing and the rest for training (LOOCV)
crossvalidation = KFold(n_splits=392, random_state=None, shuffle=False)

# cross validation scores : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
scores = cross_val_score(lm, X, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)

#print(scores)
print("Folds: " + str(len(scores)) + ", MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))


(392, 1)
Folds: 392, MSE: 24.231513517929226, STD: 36.79731503640535


In [ ]:
for i in range(1,6):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    scores = cross_val_score(lm, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)
    
    print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))


Degree-1 polynomial MSE: 24.231513517929226, STD: 36.79731503640535
Degree-2 polynomial MSE: 19.248213124489634, STD: 34.998446151782275
Degree-3 polynomial MSE: 19.334984064080896, STD: 35.76513567801413
Degree-4 polynomial MSE: 19.424430310572497, STD: 35.683352767216824
Degree-5 polynomial MSE: 19.033215741221166, STD: 35.31734430363096


# k-Fold Cross-Validation

In [ ]:
crossvalidation = KFold(n_splits=10, random_state=None, shuffle=False)

for i in range(1,11):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    scores = cross_val_score(lm, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)
    
    print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Degree-1 polynomial MSE: 27.439933652339864, STD: 14.510250711281149
Degree-2 polynomial MSE: 21.23584005580171, STD: 11.797327528897396
Degree-3 polynomial MSE: 21.336606183362797, STD: 11.844339714566736
Degree-4 polynomial MSE: 21.35388700877403, STD: 11.986332355027438
Degree-5 polynomial MSE: 20.905535162195754, STD: 12.185355430658555
Degree-6 polynomial MSE: 20.775497163172464, STD: 12.120435479212741
Degree-7 polynomial MSE: 20.953491514870567, STD: 12.060899987217143
Degree-8 polynomial MSE: 21.07728542116028, STD: 12.044640229263955
Degree-9 polynomial MSE: 21.03631569834196, STD: 11.948333581675792
Degree-10 polynomial MSE: 20.986908047797623, STD: 11.809416572672


# Which model degree will you use ?